### Import cw-simulate module and create new instance

In [ ]:
import { SimulateCosmWasmClient } from "@oraichain/cw-simulate";

senderAddress = "orai14vcw5qk0tdvknpa38wz46js5g7vrvut8lk0lk6";
client = new SimulateCosmWasmClient({
  chainId: "Oraichain",
  bech32Prefix: "orai",
  metering: true
});


### Setup account balance then transfer

In [ ]:
import { coins } from '@cosmjs/stargate';

client.app.bank.setBalance(senderAddress, coins('10000000', 'orai'));

console.log(await client.getBalance(senderAddress, 'orai'))

### Deploy contract on SimulateCosmwasmClient

In [ ]:
import { Cw20BaseClient } from "@oraichain/common-contracts-sdk";

const wasmCode = new Uint8Array(await fetch("/cw20-base.wasm").then(res=>res.arrayBuffer()));
const { codeId } = await client.upload(senderAddress, wasmCode, "auto");
const result = await client.instantiate(senderAddress, codeId, {
    decimals: 6,
    symbol: "CVT",
    name: "cosmoverse token",
    mint: { minter: senderAddress },
    initial_balances: [{ address: senderAddress, amount: "1000000000" }] 
}, "cw20 token", "auto");
contractAddress = result.contractAddress;
cw20Contract = new Cw20BaseClient(client, senderAddress, contractAddress);

console.log(await cw20Contract.tokenInfo());

### Console.log in smart contract

1. run `code ../cw-plus/contracts/cw20-base` to add log function
1. run `cwtools build ../cw-plus/contracts/cw20-base -d -w -o node_modules/@oraichain/common-contracts-build/data/` to override wasm contract file
1. re-run notebook above to see the log


### Execute contract with gas metering

In [ ]:
const { gasUsed } = await cw20Contract.transfer({amount: '1000000', recipient: 'orai1ur2vsjrjarygawpdwtqteaazfchvw4fg6uql76'});
console.log(`Transfer token required gas: ${gasUsed}`);

console.log(await cw20Contract.balance({address:'orai1ur2vsjrjarygawpdwtqteaazfchvw4fg6uql76'}))

### Load smart contract with state from running blockchain

In [ ]:
import { SortedMap } from "@oraichain/immutable";
import { BufferCollection } from "@oraichain/cw-simulate";
import { compare } from "@oraichain/cosmwasm-vm-js";
import { Cw20BaseClient } from "@oraichain/common-contracts-sdk";

const wasmCode = new Uint8Array(await fetch("/kwt-token.wasm").then(res=>res.arrayBuffer()));
const { codeId } = await client.upload(senderAddress, wasmCode, "auto");

const buffer = Buffer.from(await fetch('/kwt-token.state').then(res=>res.arrayBuffer()));
const state = SortedMap.rawPack(new BufferCollection(buffer), compare);

const kwtContract = new Cw20BaseClient(client, senderAddress, "orai1nd4r053e3kgedgld2ymen8l9yrw8xpjyaal7j5");
await client.loadContract("orai1nd4r053e3kgedgld2ymen8l9yrw8xpjyaal7j5", {
      codeId,
      admin: senderAddress,
      label: "kwt token",
      creator: senderAddress,
      created: 1,
}, state);

console.log(await kwtContract.allAccounts({limit:5}));